

# Multi-objective Optimization with Optuna

This tutorial showcases Optuna's multi-objective optimization feature by
optimizing the validation accuracy of Fashion MNIST dataset and the FLOPS of the model implemented in PyTorch.

We use [fvcore](https://github.com/facebookresearch/fvcore) to measure FLOPS.


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from fvcore.nn import FlopCountAnalysis

import optuna


DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
DIR = ".."
BATCHSIZE = 128
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features

    layers.append(nn.Linear(in_features, 10))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


# Defines training and evaluation.
def train_model(model, optimizer, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        F.nll_loss(model(data), target).backward()
        optimizer.step()


def eval_model(model, valid_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(valid_loader):
            data, target = data.view(-1, 28 * 28).to(DEVICE), target.to(DEVICE)
            pred = model(data).argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / N_VALID_EXAMPLES

    flops = FlopCountAnalysis(model, inputs=(torch.randn(1, 28 * 28).to(DEVICE),)).total()
    return flops, accuracy

Define multi-objective objective function.
Objectives are FLOPS and accuracy.



In [6]:
def objective(trial):
    train_dataset = torchvision.datasets.FashionMNIST(
        DIR, train=True, download=True, transform=torchvision.transforms.ToTensor()
    )
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_dataset, list(range(N_TRAIN_EXAMPLES))),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    val_dataset = torchvision.datasets.FashionMNIST(
        DIR, train=False, transform=torchvision.transforms.ToTensor()
    )
    val_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(val_dataset, list(range(N_VALID_EXAMPLES))),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    model = define_model(trial).to(DEVICE)

    optimizer = torch.optim.Adam(
        model.parameters(), trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    )

    for epoch in range(10):
        train_model(model, optimizer, train_loader)
    flops, accuracy = eval_model(model, val_loader)
    return flops, accuracy

## Run multi-objective optimization

If your optimization problem is multi-objective,
Optuna assumes that you will specify the optimization direction for each objective.
Specifically, in this example, we want to minimize the FLOPS (we want a faster model)
and maximize the accuracy. So we set ``directions`` to ``["minimize", "maximize"]``.



In [12]:
type(study)

optuna.study.study.Study

In [7]:
study = optuna.create_study(directions=["minimize", "maximize"])
study.optimize(objective, n_trials=30, timeout=300)

print("Number of finished trials: ", len(study.trials))

[I 2023-08-04 20:24:49,469] A new study created in memory with name: no-name-c2952ad6-422d-4571-9401-9d7769e8adf0


26422272it [00:05, 4638609.27it/s]                               


Extracting ../FashionMNIST/raw/train-images-idx3-ubyte.gz to ../FashionMNIST/raw



29696it [00:00, 10736492.68it/s]         

Extracting ../FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../FashionMNIST/raw



...

Extracting ../FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../FashionMNIST/raw

Processing...



/home/barauna/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2023-08-04 20:25:17,194] Trial 0 finished with values: [25044.0, 0.1015625] and parameters: {'n_layers': 3, 'n_units_l0': 30, 'dropout_0': 0.4446240697308833, 'n_units_l1': 38, 'dropout_1': 0.3801673204753159, 'n_units_l2': 8, 'dropout_2': 0.31923257989851883, 'lr': 1.029360567752296e-05}. 
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2023-08-04 20:25:21,947] Trial 1 finished with values: [19850.0, 0.74921875] and parameters: {'n_layers': 1, 'n_units_l0': 25, 'dropout_0': 0.3390433797958868, 'lr': 0.0013064072565525235}. 
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2023-08-04 20:25:27,375] Trial 2 finished with values: [4764.0, 0.65625] and parameters: {'n_layers': 1, 'n_units_l0': 6, 'dropout_0': 0.3579482208276602, 'lr': 0.015082031362216084}. 
Unsupported operator aten::log_softmax encountered 1 time(s)
[I 2023-08-04 20:25:33,289] Trial 3 finished with values: [20360.0, 0.22734375]

Number of finished trials:  30


Check trials on Pareto front visually.



In [8]:
optuna.visualization.plot_pareto_front(study, target_names=["FLOPS", "accuracy"])

Fetch the list of trials on the Pareto front with :attr:`~optuna.study.Study.best_trials`.

For example, the following code shows the number of trials on the Pareto front and picks the trial with the highest accuracy.



In [9]:
print(f"Number of trials on the Pareto front: {len(study.best_trials)}")

trial_with_highest_accuracy = max(study.best_trials, key=lambda t: t.values[1])
print(f"Trial with highest accuracy: ")
print(f"\tnumber: {trial_with_highest_accuracy.number}")
print(f"\tparams: {trial_with_highest_accuracy.params}")
print(f"\tvalues: {trial_with_highest_accuracy.values}")

Number of trials on the Pareto front: 5
Trial with highest accuracy: 
	number: 25
	params: {'n_layers': 1, 'n_units_l0': 65, 'dropout_0': 0.29672341031062854, 'lr': 0.0014259414835512618}
	values: [51610.0, 0.83359375]


Learn which hyperparameters are affecting the flops most with hyperparameter importance.



In [10]:
optuna.visualization.plot_param_importances(
    study, target=lambda t: t.values[0], target_name="flops"
)